In [1]:
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 25.5 MB/s eta 0:00:00


In [2]:
import flwr as fl
from flwr.common import Metrics
from flwr.common.typing import NDArrays, Scalar
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import librosa

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/daic/ProjectPrototype

/content/gdrive/MyDrive/daic/ProjectPrototype


In [5]:
!ls

'212148conf copy 3.ipynb'   edaicwoz   MFCCs_1030   MFCCs_1030.zip   preprocess_data.ipynb


In [6]:
train_labels_df = pd.read_csv("edaicwoz/train_split.csv")
test_labels_df = pd.read_csv("edaicwoz/test_split.csv")
val_labels_df = pd.read_csv("edaicwoz/dev_split.csv")

In [7]:
def load_audio_files(data_dir, sr=16000):
    file_ids = os.listdir(data_dir)
    subject_ids = []
    file_paths = []
    types = []
    labels = []
    labels_binary = []

    for file_id in file_ids:
        file_id = file_id.split("_")[0]
        file_path = [data_dir + "/" + file_id + "/" + file_id + "_MFCC_" + str(i) + ".npy" for i in range(len(next(iter(enumerate(os.walk(data_dir + "/" + str(file_id) + "/"))))[1][2]))]
        if int(file_id) in train_labels_df["Participant_ID"].values:
            types.append(0)
            labels.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])

        elif int(file_id) in test_labels_df["Participant_ID"].values:
            types.append(1)
            labels.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        else:
            types.append(2)
            labels.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        subject_ids.append(int(file_id))
        file_paths.append(file_path)

    return file_ids, subject_ids, file_paths, types, labels, labels_binary

data_dir = "MFCCs_1030"

file_ids, subject_ids, file_paths, types, labels, labels_binary = load_audio_files(data_dir)

In [8]:
def prepare_audio_set(file_paths):

    samples = []
    samples_ids = []
    samples_types = []
    samples_labels = []
    samples_labels_binary = []

    for i, file_path in enumerate(file_paths):
        all_mfccs = []
        for j in range(len(file_path)):
            all_mfccs.append(np.load(file_path[j]))
        all_mfccs = np.array(all_mfccs)
        samples.extend(all_mfccs)
        samples_ids.extend([subject_ids[i]] * len(all_mfccs))
        samples_types.extend([types[i]] * len(all_mfccs))
        samples_labels.extend([labels[i]] * len(all_mfccs))
        samples_labels_binary.extend([labels_binary[i]] * len(all_mfccs))

    samples = np.array(samples)

    samples_ids = np.array(samples_ids)
    samples_types = np.array(samples_types)
    samples_labels = np.array(samples_labels)
    samples_labels_binary = np.array(samples_labels_binary)

    return samples, samples_ids, samples_types, samples_labels, samples_labels_binary

print("[INFO] preparing data...")
samples, samples_ids, samples_types, samples_labels, samples_labels_binary = prepare_audio_set(file_paths)
samples = np.swapaxes(samples, 1, 2)

[INFO] preparing data...


In [9]:
training_samples = samples[samples_types == 0]
training_labels = samples_labels_binary[samples_types == 0]
training_subject_ids = samples_ids[samples_types == 0]

test_samples = samples[samples_types == 1]
test_labels = samples_labels_binary[samples_types == 1]

val_samples = samples[samples_types == 2]
val_labels_df = samples_labels_binary[samples_types == 2]

In [10]:
del samples, samples_ids, samples_types, samples_labels, samples_labels_binary
del file_ids, subject_ids, file_paths, types, labels, labels_binary

In [11]:
SEED = 42
NUM_CLIENTS = 4
BATCH_SIZE = 8
NUM_ROUNDS = 15
DEPRESSIVE_MULTIPLIER = 30
NON_DEPRESSIVE_MULTIPLIER = 10

In [12]:
def partition_data(X: np.ndarray, X_ids: np.ndarray, n_clients: int, d_mult: int, n_d_mult: int) -> Tuple[List[np.ndarray], List[np.ndarray]]:

    unique_subject_ids, counts = np.unique(X_ids, return_counts=True)

    # grouping the training samples by patient
    training_samples_grouped = []
    for i in unique_subject_ids:
        training_samples_grouped.append(X[X_ids == i])
    training_samples_grouped = np.array(training_samples_grouped)

    mask_30_segments = np.array([sample.shape[0] == DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    # creating masks to get deppressives and non deppressives
    mask_10_segments = np.array([sample.shape[0] == NON_DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    data_array_30_segments = training_samples_grouped[mask_30_segments]

    data_array_10_segments = training_samples_grouped[mask_10_segments]

    # recreating labels
    X_train_zeros = np.array_split(data_array_10_segments, NUM_CLIENTS)
    X_train_ones = np.array_split(data_array_30_segments, NUM_CLIENTS)

    # concatenating splitted ones and zeros with labels
    X_train_splitted = [] # (NUM_CLIENTS, data)
    y_train_splitted = [] # (NUM_CLIENTS, labels)
    for i in range(NUM_CLIENTS):

        # stack the segments from groups then concatenate
        client_data = np.concatenate((np.vstack(X_train_zeros[i]), np.vstack(X_train_ones[i])), axis=0)
        client_labels = np.concatenate((np.zeros((X_train_zeros[i].shape[0] * NON_DEPRESSIVE_MULTIPLIER), dtype=int), np.ones((X_train_ones[i].shape[0] * DEPRESSIVE_MULTIPLIER), dtype=int)), axis=0)

        X_train_splitted.append(client_data)
        y_train_splitted.append(client_labels)

    return X_train_splitted, y_train_splitted

X_trains, y_trains = partition_data(training_samples, training_subject_ids, NUM_CLIENTS, DEPRESSIVE_MULTIPLIER, NON_DEPRESSIVE_MULTIPLIER)

<ipython-input-12-45e26c792d34>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_samples_grouped = np.array(training_samples_grouped)


In [ ]:
X_trains[0].shape

(610, 15001, 13)

In [13]:
def get_model(input_shape):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.GRU(units = 64, input_shape = input_shape)) 


    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    return model

In [14]:
from flwr.common.typing import NDArrays
class FlowerClient(fl.client.NumPyClient):

    def __init__(self, model: tf.keras.models.Sequential, X_train: np.ndarray, y_train: np.ndarray):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train

    def get_parameters(self, config):
        return self.model.get_weights()


    def fit(self, parameters: NDArrays, config: Dict[str, Scalar]) -> NDArrays:

        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

        self.model.set_weights(parameters)

        history = self.model.fit(self.X_train, self.y_train ,batch_size=BATCH_SIZE, epochs=1, verbose=0)
        results = {
            "loss": history.history["loss"][0],
            "accuracy": history.history["accuracy"][0],
        }
        return self.model.get_weights(), len(self.X_train), results

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar])-> Tuple[float, int, Dict[str, Scalar]]:
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
        self.model.set_weights(parameters)

        loss, acc = self.model.evaluate(self.X_train, self.y_train, verbose=0)
        return loss, len(self.X_train), {"accuracy": acc}

In [15]:
# client creator by client id
def create_client_fn(cid: str) -> FlowerClient:

    input_shape = (15001, 13)
    model = get_model(input_shape)
    cid_int = int(cid)
    return FlowerClient(model, X_trains[cid_int], y_trains[cid_int])

In [16]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


In [17]:
patience = 5
best_accuracy = 0.0
best_loss = 999
weights = np.array([])

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    """Centralized evaluation function"""

    input_shape = (15001, 13)
    model = get_model(input_shape)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
    model.set_weights(parameters)

    loss, accuracy = model.evaluate(val_samples, val_labels_df, batch_size=16, verbose=0)

    global best_accuracy
    global best_loss
    global weights

    print(f"LOSS: {loss}")
    print(f"BEST_LOSS: {best_loss}")
    print(f"ACCURACY: {accuracy}")
    print(f"BEST_ACCURACY: {best_accuracy}")

    if loss < best_loss:
        best_accuracy = accuracy
        weights = parameters
        best_loss = loss

    return loss, {"accuracy": accuracy}



client_resources = {"num_cpus": 2}
if tf.config.get_visible_devices("GPU"):
    client_resources["num_gpus"] = 1

# Specify the Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1.0,  
    min_fit_clients=NUM_CLIENTS,  
    min_evaluate_clients=NUM_CLIENTS,  
    min_available_clients=NUM_CLIENTS,  # Wait until all 4 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
)

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=create_client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-14 12:43:13,694 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
2023-12-14 12:43:15,970	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-14 12:43:17,412 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 32532467712.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'CPU': 8.0, 'node:__internal_head__': 1.0, 'object_store_memory': 16266233856.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 32532467712.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'CPU': 8.0, 'node:__internal_head__': 1.0, 'object_store_memory': 16266233856.0}
INFO flwr 2023-12-14 12:43:17,414 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations

LOSS: 1.418653964996338
BEST_LOSS: 999
ACCURACY: 0.44999998807907104
BEST_ACCURACY: 0.0


DEBUG flwr 2023-12-14 12:45:56,994 | server.py:236 | fit_round 1 received 4 results and 0 failures
DEBUG:flwr:fit_round 1 received 4 results and 0 failures
WARNING flwr 2023-12-14 12:45:57,002 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-12-14 12:46:06,190 | server.py:125 | fit progress: (1, 0.6867510080337524, {'accuracy': 0.5274999737739563}, 141.9110378260002)
INFO:flwr:fit progress: (1, 0.6867510080337524, {'accuracy': 0.5274999737739563}, 141.9110378260002)
DEBUG flwr 2023-12-14 12:46:06,195 | server.py:173 | evaluate_round 1: strategy sampled 4 clients (out of 4)
DEBUG:flwr:evaluate_round 1: strategy sampled 4 clients (out of 4)


LOSS: 0.6867510080337524
BEST_LOSS: 1.418653964996338
ACCURACY: 0.5274999737739563
BEST_ACCURACY: 0.44999998807907104


DEBUG flwr 2023-12-14 12:46:57,489 | server.py:187 | evaluate_round 1 received 4 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 4 results and 0 failures
DEBUG flwr 2023-12-14 12:46:57,491 | server.py:222 | fit_round 2: strategy sampled 4 clients (out of 4)
DEBUG:flwr:fit_round 2: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-12-14 12:49:05,172 | server.py:236 | fit_round 2 received 4 results and 0 failures
DEBUG:flwr:fit_round 2 received 4 results and 0 failures
INFO flwr 2023-12-14 12:49:14,175 | server.py:125 | fit progress: (2, 0.6810376048088074, {'accuracy': 0.5625}, 329.8965609229999)
INFO:flwr:fit progress: (2, 0.6810376048088074, {'accuracy': 0.5625}, 329.8965609229999)
DEBUG flwr 2023-12-14 12:49:14,177 | server.py:173 | evaluate_round 2: strategy sampled 4 clients (out of 4)
DEBUG:flwr:evaluate_round 2: strategy sampled 4 clients (out of 4)


...

DEBUG flwr 2023-12-14 13:23:58,397 | server.py:187 | evaluate_round 13 received 4 results and 0 failures
DEBUG:flwr:evaluate_round 13 received 4 results and 0 failures
DEBUG flwr 2023-12-14 13:23:58,403 | server.py:222 | fit_round 14: strategy sampled 4 clients (out of 4)
DEBUG:flwr:fit_round 14: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-12-14 13:26:04,255 | server.py:236 | fit_round 14 received 4 results and 0 failures
DEBUG:flwr:fit_round 14 received 4 results and 0 failures
INFO flwr 2023-12-14 13:26:13,300 | server.py:125 | fit progress: (14, 0.697481095790863, {'accuracy': 0.5575000047683716}, 2549.021635833)
INFO:flwr:fit progress: (14, 0.697481095790863, {'accuracy': 0.5575000047683716}, 2549.021635833)
DEBUG flwr 2023-12-14 13:26:13,303 | server.py:173 | evaluate_round 14: strategy sampled 4 clients (out of 4)
DEBUG:flwr:evaluate_round 14: strategy sampled 4 clients (out of 4)


LOSS: 0.697481095790863
BEST_LOSS: 0.6801790595054626
ACCURACY: 0.5575000047683716
BEST_ACCURACY: 0.5774999856948853


DEBUG flwr 2023-12-14 13:27:03,979 | server.py:187 | evaluate_round 14 received 4 results and 0 failures
DEBUG:flwr:evaluate_round 14 received 4 results and 0 failures
DEBUG flwr 2023-12-14 13:27:03,986 | server.py:222 | fit_round 15: strategy sampled 4 clients (out of 4)
DEBUG:flwr:fit_round 15: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-12-14 13:29:12,054 | server.py:236 | fit_round 15 received 4 results and 0 failures
DEBUG:flwr:fit_round 15 received 4 results and 0 failures
INFO flwr 2023-12-14 13:29:20,981 | server.py:125 | fit progress: (15, 0.6950597167015076, {'accuracy': 0.5637500286102295}, 2736.702185647)
INFO:flwr:fit progress: (15, 0.6950597167015076, {'accuracy': 0.5637500286102295}, 2736.702185647)
DEBUG flwr 2023-12-14 13:29:20,983 | server.py:173 | evaluate_round 15: strategy sampled 4 clients (out of 4)
DEBUG:flwr:evaluate_round 15: strategy sampled 4 clients (out of 4)


LOSS: 0.6950597167015076
BEST_LOSS: 0.6801790595054626
ACCURACY: 0.5637500286102295
BEST_ACCURACY: 0.5774999856948853


DEBUG flwr 2023-12-14 13:30:08,701 | server.py:187 | evaluate_round 15 received 4 results and 0 failures
DEBUG:flwr:evaluate_round 15 received 4 results and 0 failures
INFO flwr 2023-12-14 13:30:08,703 | server.py:153 | FL finished in 2784.4247042380002
INFO:flwr:FL finished in 2784.4247042380002
INFO flwr 2023-12-14 13:30:08,705 | app.py:226 | app_fit: losses_distributed [(1, 0.6970945157903305), (2, 0.6792198774662423), (3, 0.6690468405155425), (4, 0.6599984275533798), (5, 0.6505712392482351), (6, 0.6442995664921213), (7, 0.6375612756039234), (8, 0.6280534678317131), (9, 0.621633334870034), (10, 0.6150039769233542), (11, 0.6059373264617108), (12, 0.6010795676961859), (13, 0.5869532450716546), (14, 0.5795948964484194), (15, 0.5717023897678294)]
INFO:flwr:app_fit: losses_distributed [(1, 0.6970945157903305), (2, 0.6792198774662423), (3, 0.6690468405155425), (4, 0.6599984275533798), (5, 0.6505712392482351), (6, 0.6442995664921213), (7, 0.6375612756039234), (8, 0.6280534678317131), (9, 0

In [18]:
history

History (loss, distributed):
	round 1: 0.6970945157903305
	round 2: 0.6792198774662423
	round 3: 0.6690468405155425
	round 4: 0.6599984275533798
	round 5: 0.6505712392482351
	round 6: 0.6442995664921213
	round 7: 0.6375612756039234
	round 8: 0.6280534678317131
	round 9: 0.621633334870034
	round 10: 0.6150039769233542
	round 11: 0.6059373264617108
	round 12: 0.6010795676961859
	round 13: 0.5869532450716546
	round 14: 0.5795948964484194
	round 15: 0.5717023897678294
History (loss, centralized):
	round 0: 1.418653964996338
	round 1: 0.6867510080337524
	round 2: 0.6810376048088074
	round 3: 0.6814741492271423
	round 4: 0.6815571784973145
	round 5: 0.6824567317962646
	round 6: 0.6801790595054626
	round 7: 0.6812838912010193
	round 8: 0.6851648092269897
	round 9: 0.6885053515434265
	round 10: 0.6926031708717346
	round 11: 0.6906629204750061
	round 12: 0.694620668888092
	round 13: 0.6908324956893921
	round 14: 0.697481095790863
	round 15: 0.6950597167015076
History (metrics, distributed, eval

In [19]:
# printing the validation results
print(best_accuracy)
print(best_loss)
print(patience)
print(weights)

0.5774999856948853
0.6801790595054626
5
[array([[ 0.08233454,  0.1274988 , -0.00847076, ..., -0.16272862,
         0.10172983, -0.02925962],
       [ 0.09945555, -0.13446443,  0.0205587 , ...,  0.07920102,
        -0.13699585, -0.03083093],
       [ 0.01768176,  0.12369247,  0.13807541, ...,  0.02176354,
         0.13673829,  0.04402371],
       ...,
       [ 0.04153434, -0.13687532, -0.09707322, ...,  0.16794015,
         0.09342177, -0.08677179],
       [-0.13060904, -0.12109911, -0.04314165, ..., -0.05570713,
        -0.03068337,  0.14090261],
       [-0.00243561,  0.03939537,  0.1262586 , ..., -0.06596623,
         0.04424986, -0.12256265]], dtype=float32), array([[-4.4740029e-02,  1.3858457e-02,  1.3055132e-01, ...,
        -7.9146080e-02,  1.3502184e-01, -1.1146998e-01],
       [-2.7078745e-04,  5.9276089e-02, -2.9494677e-02, ...,
         1.2541558e-01,  4.3675546e-02, -1.6591310e-03],
       [ 2.7796974e-02,  1.0665272e-01,  7.7962376e-02, ...,
         4.7348671e-02,  8.017050